# `Создание и нормализация базы данных`

<img src='data/img/pglogo.png' width=700>

____

### <a id=0>Содержание</a>

- [Нормализация базы данных](#1)
- [Структура базы данных](#2)
- [Подключение к базе данных](#3)
- [Создание таблиц](#4)
- [Загрузка данных](#5)
- [Проблемы при загрузке данных](#4)


___
## <center> <a id=1>Нормализация базы данных</a>

Изначально данные excel-файла находились в `1-й нормальной форме`, т.к.:

- все атрибуты таблицы атомарны

- все строки таблицы уникальны

- отсутствовали  ключи для таблиц

В процессе работы, база данных была приведена к `3-й нормальной форме`. Созданы таблицы, в которых каждый столбец зависит только от первичного ключа. 

|Таблица |Ключ |
|:--|:--|
|products |product_id |
| transactions|transaction_id |
| address| postcode|
|customers |customer_id |

Объекты каждой таблицы могут быть использованы отдельно от объектов других таблиц :

В таблице `address`:

- Ключом является postecode, т.к. данные столбца являются уникальными (из-за уникальности предполагаю, что в postcode заложены данные о всех географических данных, а не только почтовый индекс)

- Данные отдельны от данных клиентов, т.к. адреса не зависят от клиентов и для каждого адреса может быть определено несколько клиентов 


В таблице `customers`:

- Данные, характеризующие покупателей

В таблице `transactions`:

- Данные о покупках покупателей конкретных товаров 

В таблице `products`:

- Данные о товарах и их характеристиках





___
## <center> <a id=2>Структура базы данных</a>

<img src='data/img/sceme.png'>

[Ссылка на сайт ](https://dbdiagram.io/d)

<center>

___
## <center> <a id=3>Подключение к базе данных</a>

In [1]:
import psycopg2 as pg
from config import password


conn=pg.connect(
    dbname='transactions_customers',
        user='postgres',
          password=password,
            host='localhost',
                port='5432' )

cursor=conn.cursor()

___
## <center> <a id=4>Создание таблиц</a>

In [2]:
cursor.execute(''' 
CREATE TABLE "transactions" (
  "transaction_id" integer PRIMARY KEY,
  "product_id" integer,
  "customer_id" integer,
  "transaction_date" timestamp,
  "online_order" boolean,
  "order_status" varchar
);

CREATE TABLE "customers" (
  "customer_id" integer PRIMARY KEY,
  "first_name" varchar,
  "last_name" varchar,
  "gender" varchar,
  "DOB" timestamp,
  "job_title" varchar,
  "job_industry_category" varchar,
  "wealth_segment" varchar,
  "deceased_indicator" varchar,
  "owns_car" varchar,
  "property_valuation" integer,
  "postcode" integer
);

CREATE TABLE "address" (
  "postecode" integer PRIMARY KEY,
  "addres" varchar,
  "state" text,
  "country" varchar
);

CREATE TABLE "products" (
  "product_id" integer PRIMARY KEY,
  "brand" varchar,
  "product_line" varchar,
  "product_class" varchar,
  "product_size" varchar,
  "list_price" float,
  "standard_cost" float
);

ALTER TABLE "customers" ADD FOREIGN KEY ("postcode") REFERENCES "address" ("postecode");

ALTER TABLE "transactions" ADD FOREIGN KEY ("customer_id") REFERENCES "customers" ("customer_id");

ALTER TABLE "transactions" ADD FOREIGN KEY ("product_id") REFERENCES "products" ("product_id");

               ''')

conn.commit()


___
## <center> <a id=5>Загрузка данных</a>

Загрузка данных осуществлялась как через `DBeaver`, так и при помощи `Python`, в <a style='color:red'>**целях расширения навыков**</a> 

Приведены шаги загрузки данных для таблицы address, аналогично были загружены данные для остальных таблиц

### Загрузка в таблицу `address`:

- $\boxed{1}$ Импорт данных в таблицу при помощи контекстного меню DBeaver :

<img src='data/img/deabeaver_import/import data.png'>

- $\boxed{2}$ Выбор файла csv для загрузки :

<img src='data/img/deabeaver_import/select file.png'>

- $\boxed{3}$ Совмещение столбцов файла и столбцов таблицы при необходимости :

<img src='data/img/deabeaver_import/columns.png'>

- $\boxed{4}$ Просмотр загруженных данных :

<img src='data/img/deabeaver_import/tabledata.png'>


In [ ]:
f = open(r'data/import data/address_import.csv', 'r')
cursor.copy_from(f, 'address', sep=',')
f.close()

conn.commit()

### Загрузка в таблицу `customers`

<img src='data/img/customers.png'>

In [ ]:
f = open(r'data/import data/customers_import.csv', 'r')
cursor.copy_from(f, 'customers', sep=',')
f.close()

conn.commit()

### Загрузка в таблицу `products`

<img src='data/img/products.png'>

In [54]:
f = open(r'data/import data/product_import.csv', 'r')
cursor.copy_from(f, 'products', sep=',')
f.close()

conn.commit()


### Загрузка в таблицу `transactions`

<img src='data/img/transactions.png'>

In [61]:
f = open(r'data/import data/transaction_import.csv', 'r')
cursor.copy_from(f, 'transactions', sep=',')
f.close()

conn.commit()


___
## <center> <a id=6>Проблемы при загрузке данных</a>
- Для того, чтобы загрузить данные необходимо заполнить `пустые значения` в столбцах :

    - last_name

    - job_title

- Тип данных `boolean` больше подходит для столбцов:
    -  deceased_indicator 

    - owns_car

- Было загружено 10 объектов т.к. :

    - product_id `не уникален` для продуктов, не может быть ключом

